In [1]:
import tensorflow as tf
import scipy as sp
from scipy import ndimage
import numpy as np
import os

In [2]:
# The inputs and outptus are the maps downloaded from https://www.nnvl.noaa.gov/view/globaldata.html
# The inputs to the model are Land Surface Temperature, Moisture and Longwave Energy. 
# The output is the average precipitation map. 
# All maps are in the average weekly format. The idea is to find correlation between the input maps and 
# the output. 

input_labels = ['fire', 'frac', 'land', 'moisture', 'ndvi', 'longwave_energy'];
files = ['fire_data.txt', 'frac_data.txt', './land_data.txt', './mois_data.txt', './ndvi_data.txt', './olwr_data.txt']
input_dict = {}

xdim = 256
ydim = 512
in_chan = 6
out_chan = 1

# Process the input : input_metrics will contain a 2048 x 4096 image with 9 channels (corresponding to 3 inputs)

for label, path in zip(input_labels, files):
    
    input_dict[label] = [];
    input_dict[label] = (np.loadtxt(path,delimiter=',').reshape(-1,xdim,ydim,1))[:35,:,:,:]
    print(label, path,input_dict[label].shape)

input_metrics = input_dict[input_labels[0]];
for label in (input_labels[1:]):
    input_metrics = np.concatenate((input_metrics, input_dict[label]), axis=3);

# Process the output : output_metric will contain a 256 x 4096 image with 3 channels
path = './rain_data.txt'
output_metric = (np.loadtxt(path, delimiter=',').reshape(-1,xdim,ydim,1)[:35,:,:,:])
output_metric = np.array(output_metric);
samples = input_metrics.shape[0];

print(input_metrics.shape)
print(output_metric.shape)


fire fire_data.txt (35, 256, 512, 1)
frac frac_data.txt (35, 256, 512, 1)
land ./land_data.txt (35, 256, 512, 1)
moisture ./mois_data.txt (35, 256, 512, 1)
ndvi ./ndvi_data.txt (35, 256, 512, 1)
longwave_energy ./olwr_data.txt (35, 256, 512, 1)
(35, 256, 512, 6)
(35, 256, 512, 1)


In [3]:
# Function to obtain the next batch based on the input size and the batch size
def next_batch(indices, i):
    
    ind0, ind1 = i*batch_size, np.minimum((i+1)*batch_size, samples)
            
    return input_metrics[indices[ind0:ind1], :, :, :], output_metric[indices[ind0:ind1], :, :, :]

In [4]:
# Building a simple CNN model that looks like an auto-encoder. This is the section to change for a new model.

def conv_net(x):
    
    x = tf.reshape(x, shape=[-1, xdim, ydim, in_chan], name='reshape_x');
    x = tf.cast(x, tf.float32) 
    
    # Encoder
    # Scale down by 2x2. Out Channels = 32
    conv1 = tf.nn.relu(tf.contrib.layers.conv2d(x, 32, [8, 8], stride=2, padding='SAME', 
                                                biases_initializer=tf.zeros_initializer()))
    # Scale down by 2x2. Out Channels = 16
    conv2 = tf.nn.relu(tf.contrib.layers.conv2d(conv1, 16, [8, 8], stride=2, padding='SAME', 
                                                biases_initializer=tf.zeros_initializer()))
    # Scale down by 2x2. Out Channels = 8
    conv3 = tf.nn.relu(tf.contrib.layers.conv2d(conv2, 8, [8, 8], stride=2, padding='SAME', 
                                                biases_initializer=tf.zeros_initializer()))
    
    # Decoder
    # Scale up by 2x2. Out Channels = 16
    conv4 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv3, 16, [8, 8], stride=2, padding='SAME',
                                                         biases_initializer=tf.zeros_initializer()))
    # Scale up by 2x2. Out Channels = 32
    conv5 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv4, 32, [8, 8], stride=2, padding='SAME',
                                                         biases_initializer=tf.zeros_initializer()))
    # Scale up by 2x2. Out Channels = 1
    conv6 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv5, out_chan, [8, 8], stride=2, padding='SAME',
                                                         biases_initializer=tf.zeros_initializer()))
    
    return conv6;

Y = tf.placeholder(tf.float32, shape=(None,xdim,ydim,out_chan));
X = tf.placeholder(tf.float32, shape=[None,xdim,ydim,in_chan]);

optimizer = tf.train.AdamOptimizer(learning_rate=0.001);

out = conv_net(X);
loss_op = tf.reduce_sum(tf.multiply(Y-out,Y-out));
train_op = optimizer.minimize(loss_op);

In [5]:
num_epochs = 100;
batch_size = 8;
num_batches = int(np.ceil(float(samples) / batch_size));
print(num_batches)

with tf.Session() as sess:
    
    # Run the initializer
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        indices = np.random.permutation(samples);
        
        total_loss = 0;
        for i in range(num_batches):
            x_train, y_train = next_batch(indices, i);
            [loss, train] = sess.run([loss_op, train_op], feed_dict={X: x_train, Y: y_train});            
            total_loss += loss;
        
        print(epoch, total_loss / num_batches);

5
0 2257189376.0
1 2218280345.6
2 2137776409.6
3 2073970662.4
4 2032044697.6
5 1975707392.0
6 1939982220.8
7 1924345984.0
8 1907927168.0
9 1899823526.4
10 1881510553.6
11 1833323264.0
12 1788598630.4
13 1744103756.8
14 1712326169.6
15 1681825510.4
16 1652552038.4
17 1645062304.0
18 1620202752.0
19 1632557510.4
20 1615140787.2
21 1606552153.6
22 1574707532.8
23 1556595072.0
24 1570144665.6
25 1513407334.4
26 1477496064.0
27 1432828089.6
28 1400380134.4
29 1380496806.4
30 1238205094.4
31 1205282572.8
32 1183064454.4
33 1187127590.4
34 1179688371.2
35 1118648646.4
36 1101720390.4
37 1064130598.4
38 1050222137.6
39 1045222182.4
40 1042751846.4
41 1055499430.4
42 1033692947.2
43 1028391424.0
44 1026527782.4
45 1046282265.6
46 1068994214.4
47 1028258009.6
48 1020825504.0
49 1014691865.6
50 1007433280.0
51 1006445289.6
52 1013859833.6
53 1012879065.6
54 997363750.4
55 993524659.2
56 1029183302.4
57 1008962598.4
58 994964940.8
59 989440537.6
60 988989644.8
61 1010888614.4
62 1014126188.8
63 10